In [13]:
import requests
import random
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
# Your search parameters
keywords = "%22Data%20Engineer%22%20"
location = "United%20States"
time_filter = "r1800"  # Posted within ~30 minutes 
experience_level = "2%2C3"   #( %2C means comma in URL encoding 1=Internship, 2=Entry, 3=Associate)
start = 0 # Starting position (first page)


In [15]:
list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={keywords}&location={location}&f_TPR={time_filter}&f_E={experience_level}&start={start}"

response = requests.get(list_url)
list_data = response.text
list_soup = BeautifulSoup(list_data, 'html.parser')
page_jobs = list_soup.find_all("li")

print(list_url)
response.status_code

https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=%22Data%20Engineer%22%20&location=United%20States&f_TPR=r1800&f_E=2%2C3&start=25


200

In [16]:
id_list = []
for jobs in page_jobs:
    base_card_div = jobs.find("div", {"class": "base-card"})
    job_id = base_card_div.get("data-entity-urn").split(":")[3]
    print(job_id)
    id_list.append(job_id)

In [17]:
job_list = []

for job_id in id_list:  

    # Sleep BEFORE making the request
    time.sleep(random.uniform(0.5, 5.8))  # Random sleep to avoid being blocked
    
    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    job_response = requests.get(job_url)
    job_soup = BeautifulSoup(job_response.text, 'html.parser')
    job_post = {}
    # Extract company name
    try:
        job_post["company_name"] = job_soup.find("a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
    except AttributeError:
        job_post["company_name"] = "Not available"
    
    # Extract job title
    try:
        job_post["job_title"] = job_soup.find("h2", {"class": "top-card-layout__title"}).text.strip()
    except AttributeError:
        job_post["job_title"] = "Not available"
    
    # Extract location
    try:
        job_post["location"] = job_soup.find("span", {"class": "topcard__flavor topcard__flavor--bullet"}).text.strip()
    except AttributeError:
        job_post["location"] = "Not available"
    
    # Extract time posted
    try:
        job_post["time_posted"] = job_soup.find("span", {"class": "posted-time-ago__text"}).text.strip()
    except AttributeError:
        job_post["time_posted"] = "Not available"
    
    # Extract job description
    try:
        description_div = job_soup.find("div", {"class": "description__text description__text--rich"})
        job_post["job_description"] = description_div.text.strip()
    except AttributeError:
        job_post["job_description"] = "Not available"
    
    job_list.append(job_post)

    job_post

In [18]:
df = pd.DataFrame(job_list)

df

""
